# **Building Retrieval-Augmented Generation**

## **What's Covered?**
1. Introduction to RAG
    - Why RAG?
    - Benefits of RAG
    - RAG Building Blocks
    - Application Areas
2. How to Build a RAG System?
    - The Process
    - What is an Index?
    - What is a Retriever?
    - Index vs Retriever
3. Step 1: Create an Index on available Knowledge Base
4. Building an End-to-End RAG Chain
    - Step 1: Initialize the Chroma DB Connection
    - Step 2: Create a Retriever Object
    - Step 3: Initialize a Chat Prompt Template
    - Step 4: Initialize a Generator (i.e. Chat Model)
    - Step 5: Initialize a Output Parser
    - Step 6: Define a RAG Chain
    - Step 7: Invoke the Chain




## **Introduction to RAG**

### **Why RAG?**
We know that LLMs have the capability to generate stuff by themselves. But these tools aren't perfect.

Even though they're super smart, they sometimes get things wrong, especially if they need to be really precise or use the latest information. So, to fix this, some of the brightest minds at Meta AI came up with a new trick called retrieval-augmented generation, or RAG for short, in 2020.

Think of it as giving our language models an assistant. This assistant digs through a massive pile of updated information and feeds the most relevant and recent bits to the LLM.

### **Benefits of RAG**  
1. **Enhanced factual accuracy and Domain Specific Expertise:** Imagine a customer service chatbot trained on general conversation data. It might struggle with technical domain specific questions. RAGs allow you to integrate domain-specific knowledge bases, enabling the chatbot to handle these inquires with expertise.
2. **Reduce Hallucination:** LLMs can generate false information, a phenomenon known as hallucination. The knowledge base provided can help support the claims of generative model.

### **RAG Building Blocks**  
1. **Retrieval:** When a user asks a question or provides a prompt, retrievals first help fetch relevant passages from a vast knowledge base. This Knowledge Base could be the company's internal documents, or any other source of text data.
2. **Augmentation:** The retrieved passages are then used to "augment" the LLM's knowledge. This can include various techniques, such as summarization or encoding the key information.
3. **Generation:** Finally LLM leverages its understanding of language along with the augmented information to generate a response. This response can be an answer to a question, a creative text format based on a prompt, etc...

### **Application Areas**  
1. Question Answering: A RAG powered customer care chatbot can answer customer queries by retrieving product information, FAQs and guides to provide a well-rounded response.
2. Document Summarization: A research paper summarization tool can use RAG to retrieve relevant sections and then generate a summary highlighting main points.
3. Creative Text Generation: A story writing assistant can use RAG to retrieve information about historical periods or fictional creation, helping LLM to generate more deeply engaging stories.
4. Code Generation: A code completion tool can use RAG to retrieve relevant code examples and API documentation, helping developers write code more efficiently.

## **How to Build a RAG System?**

### **The Process**
**Step 1: Create an Index on available Knowledge Base i.e. the preprocessing strategy**
- This step is about preprocessing and organizing the raw data so it can be searched efficiently later.
- Data from formats like PDF, HTML, etc is cleaned and converted into plain text. This text is then divided into smaller parts (i.e chunks) and turned into vector representations by passing the chunks into the embedding model to make it easier to find later.
- **Key Outcome:** You now have a vector store that contains all chunks from your knowledge base in vector format.

**Step 2: Create a Retriever i.e. the search strategy**
- This step defines how you want to search inside the index at query time. It does not store embeddings or build new indexes—it only queries the index.
- When someone asks a question, the Retrieval system turns that question into vector embedding using the same method used in indexing. Then, it compares this vector to the vectors of the indexed text parts to fing the `k` most similar chunks. These `k` most similar chunks are used in the next step as a context.
- You must specify:
    - How many results to return (k)
    - What similarity metric to use (eucledian, cosine, etc...)
    - Whether to use MMR, hybrid search, metadata filters, reranking, etc...
- **Key Outcome:** This stage is real-time and repeated every query. When the user asks a question, the retriever:
    - embeds the question
    - runs similarity search on the existing index
    - returns top relevant chunks

**Step 3: Generation**
- The system combines the retrieved text parts (i.e. context) with the original question to create a prompt. The LLM uses this prompt to answer the question.

### **What is an Index?**
Indexing is the process of converting raw data (text, PDFs, docs, JSON, webpages, etc.) into vector-encoded searchable chunks, and storing those representations in a database optimized for similarity search. 

Without indexing, you would have to search through raw text every time, which is:
- slow,
- expensive,
- inaccurate,
- not scalable.

### **What is a Retriever?**
Understand that the retrievals are specialized in navigating through vast amounts of data to find information that is relevant to a specific query or context. 

Retrieval models focus on the precision of matching query criteria with the data they have access to. Note that retrieval models rely heavily on the quality and structure of the data they access. Their performance depends on the relevance and accuracy of the information stored in the databases they query. 

In simple terms, retrievals search and identify relevant data from a large corpus for a given query.

### **Index vs Retriever**
- **Indexing** = preprocessing and storage
- **Retrieval** = query-time search using the index

**Index is:**
- Built once (or periodically)
- Heavy processing
- Offline step

**Retriever is:**
- Runs per user query
- Uses similarity search over the index
- Lightweight, real-time

**Important: An Index makes your knowledge base searchable by meaning, not keywords, enabling fast and accurate retrieval in RAG systems.**

In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

loader = DirectoryLoader('data/subtitles', glob="*.srt", show_progress=True, loader_cls=TextLoader)

docs = loader.load()

/Users/kanavbansal/Developer/.env_langchain/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|█████████████████████████████████████████| 10/10 [00:00<00:00, 5726.79it/s]


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

chunks = text_splitter.split_documents(docs)

In [3]:
print("Number of Documents:", len(docs))
print()
print("Number of Chunks:", len(chunks))

Number of Documents: 10

Number of Chunks: 514


In [4]:
# Initialize a ChromaDB Connection
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

f = open("keys/.openai_api_key.txt")
OPENAI_API_KEY = f.read()
embedding_model = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

# Initialize the database connection
# If database exist, it will connect with the collection_name and persist_directory
# Otherwise a new collection will be created
db = Chroma(collection_name="vector_database", 
            embedding_function=embedding_model, 
            persist_directory="./chroma_db_")

In [5]:
db.add_documents(documents=chunks)

['29cef22f-3faf-4c6e-bfe1-1dfaec6409ad',
 'bbe0eb29-35ac-46f9-8939-635e130060d6',
 '8fe9b1ed-0b68-427d-81b1-e4fc9258fab9',
 'b3f984ea-1361-4eb1-8e14-10ff199dd09b',
 '51a4abf5-43c7-48f5-bd96-a79fe4d3c275',
 'd2132276-f5a8-4405-a3e1-279982972db5',
 '7294fe69-8f4a-488a-be68-fbd6f8dba625',
 '93d45284-d54c-4dc0-bdcf-a48581f474f5',
 '88df4f05-fdd0-45b6-8cfd-1c57340d637b',
 'e5bd38a2-dcb4-4bfc-912c-c3b8ccfa3142',
 'dda52273-f844-4989-91d9-833d451a9460',
 '23e949e2-5f6e-46cb-a6ac-413fff73314a',
 '8496653a-b8e4-4698-917d-ccc4ffe6972e',
 'd5fd1442-df15-43f0-9129-ca7520311474',
 '4820f90c-cd8b-4e6a-bfbe-d6225e252423',
 'b4f82f11-9944-4277-b177-dcd8ecbeabed',
 '596b3d9e-2282-48cf-ad7d-0d0b3910122f',
 '89e2b57e-8609-4fbd-baba-d15e1770ea24',
 '8b620f13-664a-4823-b351-b798deea0324',
 'f1b870a4-e829-43da-b4cd-dc527986f85e',
 'c349c8e8-d62c-4a7a-8236-8f2135a8bd27',
 '1f86e226-a216-4ddf-bd7d-ac03c1b334ed',
 '95680beb-f567-4b4f-958e-1224d3a5aaf4',
 '04ccc1be-30e6-4961-b57e-f1ce4a3c9a29',
 'c3523151-ad9b-

In [6]:
# We can check the already existing values
print(len(db.get()["ids"]))

514


## **Building an End-to-End RAG Chain**

**Step 1: Initialize the Chroma DB Connection**  
**Step 2: Create a Retriever Object**   
**Step 3: Initialize a Chat Prompt Template**  
**Step 4: Initialize a Generator (i.e. Chat Model)**  
**Step 5: Initialize a Output Parser**   
**Step 6: Define a RAG Chain**  
**Step 7: Invoke the Chain**

In [7]:
# Step 1: Initialize the Chroma DB Connection

from langchain_chroma import Chroma

# Initialize the database connection
# If database exist, it will connect with the collection_name and persist_directory
# Otherwise a new collection will be created
db = Chroma(collection_name="vector_database", 
            embedding_function=embedding_model, 
            persist_directory="./chroma_db_")

# We can check the already existing values
print(len(db.get()["ids"]))

514


In [8]:
# Step 2: Create a Retriever Object 

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [9]:
# Step 3: Initialize a Chat Prompt Template

from langchain_core.prompts import ChatPromptTemplate

PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

In [10]:
# Step 4: Initialize a Generator (i.e. Chat Model)

from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [11]:
# Step 5: Initialize a Output Parser

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [12]:
# Step 6: Define a RAG Chain

from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)



In [16]:
rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt_template | chat_model | parser

In [18]:
# Invoke the Chain

query = 'Who is Rachem?'

rag_chain.invoke(query)

'Rachem is not a real word or name. It seems to be a misunderstanding or a mispronunciation of the name Rachel. The speaker is correcting someone who seems to have mistakenly referred to Rachel as "Rachem" or possibly making a joke about the name.'

In [19]:
# Invoke the Chain

query = 'What is there on the List comparing Rachel and Julie?'

rag_chain.invoke(query)

'The list compares Rachel and Julie in terms of their qualities or characteristics. The list mentions "She\'s not Rachel." This implies that there are differences between Julie and Rachel that are being highlighted.'